

---

# Nazaraz: Your Personal Calendar Optimizer

Nazaraz is a tool designed to revolutionize the way you manage your personal calendar. Using advanced machine learning techniques and rule-based expert systems, Nazaraz easily analyzes your past schedule, understands your preferences, and suggests the most optimal time slots for new events. With a focus on simplicity, efficiency, and personalization, Nazaraz ensures that your calendar works for you, not the other way around.

In [ ]:
import pandas as pd
import numpy as np
def transform_to_full_events_with_handling(data):
    """
    Transforms a dataset of respondents' daily schedules into full events,
    converting start and stop times to minutes and handling missing values.

    Args:
        data (pd.DataFrame): Original dataset containing 'Wir', 'type_activity', 'duration',
                             'start_time', 'stop_time', and potentially 'priority'.

    Returns:
        list: A list of DataFrames, each representing a respondent's day as a sequence of full events.
    """

    data["start_time_minutes"] = (
        pd.to_datetime(data["start_time"]).dt.hour * 60 +
        pd.to_datetime(data["start_time"]).dt.minute
    )

    data["start_time_minutes"] = data["start_time_minutes"].apply(
        lambda x: x if x >= 240 else x + 1440
    )

    data["stop_time_minutes"] = data["start_time_minutes"].astype(int) + data["duration"].astype(int)

    respondent_events = []
    for respondent_id, group in data.groupby("ID"):
        group = group.sort_values("start_time_minutes").reset_index(drop=True)

        group["priority"] = group["priority"].fillna(16)
        group["type_activity"] = group["type_activity"].fillna("Unknown")
        group["type_activity_detailed"] = group["type_activity_detailed"].fillna("None")

        respondent_events.append(group)

    return respondent_events

df_new = pd.read_csv('data_cleaned.csv')
# transformed_events = transform_to_full_events_with_handling(df_new[0:10000])
transformed_events = transform_to_full_events_with_handling(df_new)

<ipython-input-35-5a1abda07153>:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(data["start_time"]).dt.hour * 60 +
<ipython-input-35-5a1abda07153>:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(data["start_time"]).dt.minute


In [ ]:
transformed_events[0]

,ID,type_activity,duration,start_time,stop_time,org_ID,org_ID_4,org_ID_2,type_activity_detailed,priority,start_time_minutes,stop_time_minutes
0,20030100013280,"Sports, Exercise, & Recreation",60,04:00,05:00,130124,1301,13,"Participating in Sports, Exercise, and Recreation",5,240,300
1,20030100013280,Personal Care Activities,30,05:00,05:30,10201,102,1,Grooming,1,300,330
2,20030100013280,Personal Care Activities,600,05:30,15:30,10101,101,1,Sleeping,1,330,930
3,20030100013280,"Socializing, Relaxing, and Leisure",150,15:30,18:00,120303,1203,12,Relaxing and Leisure,4,930,1080
4,20030100013280,Eating and Drinking,5,18:00,18:05,110101,1101,11,Eating and Drinking,2,1080,1085
5,20030100013280,"Socializing, Relaxing, and Leisure",175,18:05,21:00,120303,1203,12,Relaxing and Leisure,4,1085,1260
6,20030100013280,Personal Care Activities,270,21:00,01:30,10101,101,1,Sleeping,1,1260,1530
7,20030100013280,Personal Care Activities,10,01:30,01:40,10201,102,1,Grooming,1,1530,1540
8,20030100013280,"Sports, Exercise, & Recreation",140,01:40,04:30,130124,1301,13,"Participating in Sports, Exercise, and Recreation",5,1540,1680


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def prepare_training_data_lstm_events(transformed_events):
    """
    Prepares data for an LSTM model based on full events instead of 15-minute time blocks.

    Args:
        transformed_events (list of pd.DataFrame): List of DataFrames, each representing a day's events.

    Returns:
        tuple: X (features), y (labels) for the LSTM model.
    """
    X, y = [], []

    for day_events in transformed_events:
        day_events = day_events.reset_index(drop=True)
        sequence = []
        labels = []

        for index, event in day_events.iterrows():
            features = [
                int(event["start_time_minutes"]),
                int(event["duration"]),
                int(event["priority"]) if pd.notna(event["priority"]) else 16,
                1 if event["type_activity"] else 0,

                # Time of Day Features
                1 if 4 * 60 <= event["start_time_minutes"] < 9 * 60 else 0,
                1 if 9 * 60 <= event["start_time_minutes"] < 18 * 60 else 0,
                1 if 18 * 60 <= event["start_time_minutes"] < 22 * 60 else 0,
                1 if event["start_time_minutes"] >= 22 * 60 or event["start_time_minutes"] < 4 * 60 else 0,

                # Fraction of day
                event["start_time_minutes"] / (24 * 60),
            ]

            time_of_day_score = (
                2 if 9 * 60 <= event["start_time_minutes"] <= 18 * 60 else
                -2 if event["start_time_minutes"] < 6 * 60 or event["start_time_minutes"] > 22 * 60 else
                0
            )

            priority_score = -int(event["priority"]) if pd.notna(event["priority"]) else -16

            unique_activities = len(day_events["type_activity"].unique())
            diversity_score = 2 if unique_activities > 5 else 0

            if index > 0:
                previous_event_end = day_events.loc[index - 1, "stop_time_minutes"]
                idle_time = int(event["start_time_minutes"]) - int(previous_event_end)
                idle_penalty = -1 if idle_time > 30 else 0
            else:
                idle_penalty = 0

            total_day_duration = int(day_events["duration"].sum())
            density_score = (
                2 if total_day_duration > (8 * 60) else
                -2 if total_day_duration < (4 * 60) else
                0
            )

            score = (
                time_of_day_score +
                priority_score +
                diversity_score +
                idle_penalty +
                density_score
            )

            sequence.append(features)
            labels.append(score)

        if sequence:
            X.append(sequence)
            y.append(labels)

    X_padded = pad_sequences(X, padding="post", dtype="float32")
    y_padded = pad_sequences(y, padding="post", dtype="float32")

    X_padded = np.nan_to_num(X_padded, nan=0.0)
    y_padded = np.nan_to_num(y_padded, nan=0.0)

    return X_padded, y_padded


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Dropout
from tensorflow.keras.optimizers import Adam

def build_lstm_model(input_shape):
    """
    Builds an LSTM model for scoring time slots.

    Args:
        input_shape (tuple): Shape of the input data (timesteps, features).

    Returns:
        model: Compiled LSTM model.
    """
    model = Sequential()
    model.add(Masking(mask_value=0.0, input_shape=input_shape))
    model.add(LSTM(64, return_sequences=True, activation="tanh"))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss="mse", metrics=["mae"])
    return model

In [ ]:
X, y = prepare_training_data_lstm_events(transformed_events)
y = y.reshape(y.shape[0], y.shape[1], 1)

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (49936, 91, 9)
Shape of y: (49936, 91, 1)


In [ ]:
input_shape = (X.shape[1], X.shape[2])
lstm_model = build_lstm_model(input_shape)

lstm_model.fit(X, y, epochs=20, batch_size=16, validation_split=0.2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
2497/2497 ━━━━━━━━━━━━━━━━━━━━ 143s 56ms/step - loss: 5.5591 - mae: 1.3556 - val_loss: 0.5950 - val_mae: 1.1157
Epoch 2/20
2497/2497 ━━━━━━━━━━━━━━━━━━━━ 143s 57ms/step - loss: 1.0944 - mae: 1.1421 - val_loss: 0.5571 - val_mae: 0.9589
Epoch 3/20
2497/2497 ━━━━━━━━━━━━━━━━━━━━ 201s 57ms/step - loss: 0.9046 - mae: 1.1390 - val_loss: 0.6435 - val_mae: 1.4325
Epoch 4/20
2497/2497 ━━━━━━━━━━━━━━━━━━━━ 140s 56ms/step - loss: 0.8094 - mae: 1.1415 - val_loss: 0.6013 - val_mae: 1.1471
Epoch 5/20
2497/2497 ━━━━━━━━━━━━━━━━━━━━ 142s 57ms/step - loss: 0.7370 - mae: 1.1419 - val_loss: 0.7103 - val_mae: 1.2252
Epoch 6/20
2497/2497 ━━━━━━━━━━━━━━━━━━━━ 139s 56ms/step - loss: 0.6992 - mae: 1.1468 - val_loss: 0.7348 - val_mae: 0.8340
Epoch 7/20
2497/2497 ━━━━━━━━━━━━━━━━━━━━ 142s 56ms/step - loss: 0.6669 - mae: 1.1321 - val_loss: 0.8293 - val_mae: 1.0358
Epoch 8/20
2497/2497 ━━━━━━━━━━━━━━━━━━━━ 138s 55ms/step - loss: 0.6592 - mae: 1.1426 - val_loss: 0.6927 - val_mae: 1.0040
Epoch 9/20
2497/

# adding new event to day schedule

In [ ]:
day_for_test = pd.read_excel("day_for_test.xls")
day_for_test = day_for_test.dropna()
day_for_test

,ID,type_activity,duration,start_time,stop_time,org_ID,org_ID_4,org_ID_2,type_activity_detailed,priority
0,2.003111e+13,Personal Care Activities,225.0,04:00,07:45,10101.0,101.0,1.0,Sleeping,1.0
1,2.003111e+13,Caring For & Helping Household (HH) Members,45.0,07:45,08:30,30101.0,301.0,3.0,Caring For & Helping HH Children,11.0
2,2.003111e+13,Traveling,5.0,08:30,08:35,180381.0,1803.0,18.0,Travel Related to Caring For & Helping Househo...,10.0
3,2.003111e+13,Caring For & Helping Household (HH) Members,5.0,08:35,08:40,30112.0,301.0,3.0,Caring For & Helping HH Children,11.0
8,2.003111e+13,Work & Work-Related Activities,275.0,11:55,16:30,50101.0,501.0,5.0,Working,3.0
9,2.003111e+13,Traveling,15.0,16:30,16:45,180701.0,1807.0,18.0,Travel Related to Consumer Purchases,10.0
10,2.003111e+13,Consumer Purchases,30.0,16:45,17:15,70101.0,701.0,7.0,"Shopping (Store, Telephone, Internet)",12.0
15,2.003111e+13,Traveling,15.0,18:16,18:31,181201.0,1812.0,18.0,"Travel Related to Socializing, Relaxing, and L...",10.0
16,2.003111e+13,"Socializing, Relaxing, and Leisure",5.0,18:31,18:36,120101.0,1201.0,12.0,Socializing and Communicating,4.0
17,2.003111e+13,Data Codes,90.0,18:36,20:06,500104.0,5001.0,50.0,Unable to Code,8.0


In [ ]:
def transform_to_15_min_blocks_with_priority_and_nan_handling(data):
    """
    Transforms a dataset of respondents' daily schedules into 15-minute time blocks.
    Handles missing values (NaNs) in 'type_activity' and supports future columns like 'priority'.

    Args:
        data (pd.DataFrame): Original dataset containing 'Wir', 'type_activity', 'duration',
                             'start_time', 'stop_time', and potentially 'priority'.

    Returns:
        list: A list of DataFrames, each representing a respondent's day transformed into 15-minute blocks.
    """

    data["start_time_minutes"] = (
    pd.to_datetime(data["start_time"]).dt.hour * 60 +
    pd.to_datetime(data["start_time"]).dt.minute
    )


    data["start_time_minutes"] = data["start_time_minutes"].apply(
        lambda x: x if x >= 240 else x + 1440
    )

    data["stop_time_minutes"] = data["start_time_minutes"].astype(int) + data["duration"].astype(int)


    respondent_blocks = []


    for respondent_id, group in data.groupby("ID"):
        day_split = []
        for block in range(4 * 60, 24 * 60 + 4 * 60, 15):
            block_dict = {
                "block_start_time": block,
                "block_end_time": block + 15,
                "is_available": True,
                "Wir": respondent_id,
                "type_activity": None,
                "priority": None,
                "type_activity_detailed": None
            }

            for _, row in group.iterrows():
                event_start = row["start_time_minutes"]
                event_end = row["stop_time_minutes"]

                if event_start < block + 15 and event_end > block:
                    block_dict.update({
                        "type_activity": row["type_activity"] if pd.notna(row["type_activity"]) else "Unknown",
                        "priority": row.get("priority", None),
                        "is_available": False,
                        "type_activity_detailed": row.get("type_activity_detailed", None)
                    })
                    break

            day_split.append(block_dict)

        respondent_blocks.append(pd.DataFrame(day_split))

    return respondent_blocks

In [ ]:
data_for_add = transform_to_15_min_blocks_with_priority_and_nan_handling(day_for_test)

<ipython-input-42-b4796dab71e2>:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(data["start_time"]).dt.hour * 60 +
<ipython-input-42-b4796dab71e2>:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(data["start_time"]).dt.minute


In [ ]:
data_for_add[0]

,block_start_time,block_end_time,is_available,Wir,type_activity,priority,type_activity_detailed
0,240,255,False,2.003111e+13,Personal Care Activities,1.0,Sleeping
1,255,270,False,2.003111e+13,Personal Care Activities,1.0,Sleeping
2,270,285,False,2.003111e+13,Personal Care Activities,1.0,Sleeping
3,285,300,False,2.003111e+13,Personal Care Activities,1.0,Sleeping
4,300,315,False,2.003111e+13,Personal Care Activities,1.0,Sleeping
...,...,...,...,...,...,...,...
91,1605,1620,False,2.003111e+13,Personal Care Activities,1.0,Sleeping
92,1620,1635,False,2.003111e+13,Personal Care Activities,1.0,Sleeping
93,1635,1650,False,2.003111e+13,Personal Care Activities,1.0,Sleeping
94,1650,1665,False,2.003111e+13,Personal Care Activities,1.0,Sleeping


In [ ]:
def transform_single_event_to_duration_blocks(event_row, block_duration=15):
    """
    Transforms a single event into blocks based on its duration.

    Args:
        event_row (pd.Series): A single row of event data containing 'duration', 'type_activity', and other fields.
        block_duration (int): The duration of each time block in minutes (default: 15 minutes).

    Returns:
        pd.DataFrame: A DataFrame representing the event split into blocks based on duration.
    """
    total_blocks = int(event_row["duration"]) // block_duration

    blocks = []
    for _ in range(total_blocks):
        block_dict = {
            "is_available": False,
            "Wir": event_row.get("ID", None),
            "type_activity": event_row.get("type_activity", "Unknown"),
            "priority": event_row.get("priority", None),
            "type_activity_detailed": event_row.get("type_activity_detailed", None)
        }
        blocks.append(block_dict)


    return pd.DataFrame(blocks)

event_to_add = pd.DataFrame({
    "ID": ["20031110032037"],
    "type_activity": ["Data Codes"],
    "duration": ["60"],
    "type_activity_detailed":[ "Unable to Code"],
    "priority": 8.0

    })

transform_single_event_to_duration_blocks(event_to_add.iloc[0])

,is_available,Wir,type_activity,priority,type_activity_detailed
0,False,20031110032037,Data Codes,8.0,Unable to Code
1,False,20031110032037,Data Codes,8.0,Unable to Code
2,False,20031110032037,Data Codes,8.0,Unable to Code
3,False,20031110032037,Data Codes,8.0,Unable to Code


In [ ]:
def add_event_to_schedule_by_duration(all_day_schedule, new_event, block_duration=15):
    """
    Adds a new event to all possible places in an all-day schedule based only on duration,
    and returns an array of possible schedules.

    Args:
        all_day_schedule (pd.DataFrame): The current day's schedule split into blocks.
        new_event (pd.Series): The new event to be added.
        block_duration (int): The duration of each time block in minutes (default: 15 minutes).

    Returns:
        list: A list of DataFrames, each representing a possible schedule with the new event added.
    """
    new_event_blocks = transform_single_event_to_duration_blocks(new_event, block_duration)

    new_event_duration_blocks = len(new_event_blocks)

    possible_schedules = []

    for i in range(len(all_day_schedule) - new_event_duration_blocks + 1):
        can_fit = True
        for j in range(new_event_duration_blocks):
            if not all_day_schedule.iloc[i + j]["is_available"]:
                can_fit = False
                break

        if can_fit:
            new_schedule = all_day_schedule.copy()
            print(i)
            for j in range(new_event_duration_blocks):
                new_schedule.loc[i + j, new_event_blocks.columns] = new_event_blocks.iloc[j].values

            possible_schedules.append(new_schedule)

    return possible_schedules



new_event_example = event_to_add.iloc[0]

possible_schedules = add_event_to_schedule_by_duration(data_for_add[0], new_event_example)
possible_schedules

19
20
21
22
23
24
25
26
27
53


<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20031110032037' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc[i + j, new_event_blocks.columns] = new_event_blocks.iloc[j].values
<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20031110032037' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc[i + j, new_event_blocks.columns] = new_event_blocks.iloc[j].values
<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20031110032037' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc

[    block_start_time  block_end_time  is_available               Wir  \
 0                240             255         False  20031110032037.0   
 1                255             270         False  20031110032037.0   
 2                270             285         False  20031110032037.0   
 3                285             300         False  20031110032037.0   
 4                300             315         False  20031110032037.0   
 ..               ...             ...           ...               ...   
 91              1605            1620         False  20031110032037.0   
 92              1620            1635         False  20031110032037.0   
 93              1635            1650         False  20031110032037.0   
 94              1650            1665         False  20031110032037.0   
 95              1665            1680         False  20031110032037.0   
 
                type_activity  priority type_activity_detailed  
 0   Personal Care Activities       1.0               Sle

In [ ]:
def convert_schedules(possible_schedules, block_duration=15):
    corrected_schedules = []

    for schedule in possible_schedules:
        events = []
        current_event = None

        for _, block in schedule.iterrows():
            if not block["is_available"]:
                if (
                    current_event is None
                    or block["block_start_time"] != current_event["stop_time_minutes"]
                    or block["type_activity"] != current_event["type_activity"]
                ):
                    if current_event:
                        events.append(current_event)

                    current_event = {
                        "ID": block["Wir"],
                        "type_activity": block["type_activity"],
                        "type_activity_detailed": block["type_activity_detailed"],
                        "start_time_minutes": block["block_start_time"],
                        "stop_time_minutes": block["block_end_time"],
                        "duration": block_duration,
                        "priority": block["priority"],
                    }
                else:
                    current_event["stop_time_minutes"] = block["block_end_time"]
                    current_event["duration"] += block_duration

        if current_event:
            events.append(current_event)

        corrected_schedules.append(pd.DataFrame(events))

    return corrected_schedules


In [ ]:
import numpy as np

def evaluate_schedules_with_model(possible_schedules, model):
    """
    Evaluates all possible schedules with the trained LSTM model.

    Args:
        possible_schedules (list): A list of DataFrames, each representing a possible schedule.
        model: Trained LSTM model.
        event_duration (int): Duration of the new event in minutes.

    Returns:
        list: A list of tuples (schedule_index, score), ranked by score.
    """
    schedule_scores = []

    for idx, schedule in enumerate(possible_schedules):
        transformed_blocks = [schedule]
        X, _ = prepare_training_data_lstm_events(transformed_blocks)

        predictions = model.predict(X)
        predictions = predictions.squeeze()

        new_event_indices = [
            i for i, block in enumerate(schedule["type_activity"])
        ]
        new_event_score = sum(predictions[new_event_indices]) / len(new_event_indices)

        schedule_scores.append((idx, new_event_score))

    ranked_schedules = sorted(schedule_scores, key=lambda x: x[1], reverse=True)
    return ranked_schedules

converted_schedules = convert_schedules(possible_schedules)
ranked_schedules = evaluate_schedules_with_model(converted_schedules, lstm_model)

best_schedule_index, best_score = ranked_schedules[0]
print(f"Best Schedule Index: {best_schedule_index}, Score: {best_score}")

best_schedule = converted_schedules[best_schedule_index]
best_schedule

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Best Schedule Index: 8, Score: -3.1832977625039907


,ID,type_activity,type_activity_detailed,start_time_minutes,stop_time_minutes,duration,priority
0,20031110032037.0,Personal Care Activities,Sleeping,240,465,225,1.0
1,20031110032037.0,Caring For & Helping Household (HH) Members,Caring For & Helping HH Children,465,510,45,11.0
2,20031110032037.0,Traveling,Travel Related to Caring For & Helping Househo...,510,525,15,10.0
3,20031110032037,Data Codes,Unable to Code,645,705,60,8.0
4,20031110032037.0,Work & Work-Related Activities,Working,705,990,285,3.0
5,20031110032037.0,Traveling,Travel Related to Consumer Purchases,990,1005,15,10.0
6,20031110032037.0,Consumer Purchases,"Shopping (Store, Telephone, Internet)",1005,1035,30,12.0
7,20031110032037.0,Traveling,"Travel Related to Socializing, Relaxing, and L...",1095,1125,30,10.0
8,20031110032037.0,Data Codes,Unable to Code,1125,1215,90,8.0
9,20031110032037.0,Traveling,Travel Related to Consumer Purchases,1245,1275,30,10.0


# Ще один день

In [ ]:
second_event = pd.DataFrame({
    "ID": ["20050808050100"],
    "type_activity": ["Education"],
    "duration": ["60"],
    "type_activity_detailed":[ "Research/Homework"],
    "priority": [6.0]
    })

second_day = pd.read_excel("second_day.xls")
second_day = second_day.dropna()
second_day = transform_to_15_min_blocks_with_priority_and_nan_handling(second_day)[0]

<ipython-input-42-b4796dab71e2>:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(data["start_time"]).dt.hour * 60 +
<ipython-input-42-b4796dab71e2>:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(data["start_time"]).dt.minute


In [ ]:
second_day

,block_start_time,block_end_time,is_available,Wir,type_activity,priority,type_activity_detailed
0,240,255,True,2.005081e+13,None,NaN,None
1,255,270,True,2.005081e+13,None,NaN,None
2,270,285,True,2.005081e+13,None,NaN,None
3,285,300,True,2.005081e+13,None,NaN,None
4,300,315,True,2.005081e+13,None,NaN,None
...,...,...,...,...,...,...,...
91,1605,1620,True,2.005081e+13,None,NaN,None
92,1620,1635,True,2.005081e+13,None,NaN,None
93,1635,1650,True,2.005081e+13,None,NaN,None
94,1650,1665,True,2.005081e+13,None,NaN,None


In [ ]:
second_possibles = add_event_to_schedule_by_duration(second_day, second_event.iloc[0])


0
1

<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20050808050100' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc[i + j, new_event_blocks.columns] = new_event_blocks.iloc[j].values
<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20050808050100' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc[i + j, new_event_blocks.columns] = new_event_blocks.iloc[j].values
<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20050808050100' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc


2
3
4
5
6
7
8
14
28
29
30
31
32
55


<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20050808050100' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc[i + j, new_event_blocks.columns] = new_event_blocks.iloc[j].values
<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20050808050100' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc[i + j, new_event_blocks.columns] = new_event_blocks.iloc[j].values
<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20050808050100' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc

56


...

<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20050808050100' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc[i + j, new_event_blocks.columns] = new_event_blocks.iloc[j].values
<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20050808050100' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc[i + j, new_event_blocks.columns] = new_event_blocks.iloc[j].values
<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20050808050100' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc

85


<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20050808050100' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc[i + j, new_event_blocks.columns] = new_event_blocks.iloc[j].values
<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20050808050100' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc[i + j, new_event_blocks.columns] = new_event_blocks.iloc[j].values
<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20050808050100' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc

86
87
88
89
90
91
92


<ipython-input-46-9215fa4de1e1>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20050808050100' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_schedule.loc[i + j, new_event_blocks.columns] = new_event_blocks.iloc[j].values


In [ ]:
converted_schedules = convert_schedules(second_possibles)
ranked_schedules = evaluate_schedules_with_model(converted_schedules, lstm_model)

best_schedule_index, best_score = ranked_schedules[0]
print(f"Best Schedule Index: {best_schedule_index}, Score: {best_score}")

best_schedule = converted_schedules[best_schedule_index]
best_schedule

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━

,ID,type_activity,type_activity_detailed,start_time_minutes,stop_time_minutes,duration,priority
0,20050808050100.0,Eating and Drinking,Eating and Drinking,420,450,30,2.0
1,20050808050100.0,Traveling,Travel related to religious/spiritual practices,510,540,30,10.0
2,20050808050100.0,Religious and Spiritual Activities,Religious/Spiritual Practices,540,630,90,9.0
3,20050808050100.0,Education,Research/Homework,630,660,30,6.0
4,20050808050100,Education,Research/Homework,675,735,60,6.0
5,20050808050100.0,Eating and Drinking,Eating and Drinking,780,840,60,2.0
6,20050808050100.0,Household Activities,Housework,870,885,15,13.0
7,20050808050100.0,"Socializing, Relaxing, and Leisure",Relaxing and Leisure,885,1065,180,4.0
8,20050808050100.0,Traveling,Travel related to religious/spiritual practices,1140,1215,75,10.0
9,20050808050100.0,Household Activities,Housework,1215,1320,105,13.0
